## 7 AIS dataset process - Encounter generator (Not considered towards thesis scope)

In [151]:
# from sklearn.neighbors import BallTree
import numpy as np
import pandas as pd
from haversine import haversine, Unit

pd.options.mode.chained_assignment = None  # default='warn'

In [152]:
to_read_own_vessel_path = r'W:\Neeraj_thesis\version2_thesis\dataset\2019\generator_version_2_output_unifom_ts\data_2019_02_v2_uniform_ts_out.csv'
to_read_other_vessel_path = r'W:\Neeraj_thesis\version2_thesis\dataset\2019\generator_version_6_output_uniform_ts\data_2019_02_v6_out_uniform_ts.csv'
to_store_path =  r'W:\Neeraj_thesis\version2_thesis\dataset\2019\generator_version_10_output_encounter_unifom_ts\data_2019_02_v10_out_encounters_uniform_ts.csv'

In [153]:
ais_own_data = pd.read_csv(to_read_own_vessel_path)
ais_other_data = pd.read_csv(to_read_other_vessel_path)

In [154]:
ais_encounter_data = ais_own_data.copy()

In [155]:
for i in range(1, 11):
    ais_encounter_data[f'{i}_encounter_vessel_mmsi'] = None
    ais_encounter_data[f'{i}_encounter_vessel_lat'] = None
    ais_encounter_data[f'{i}_encounter_vessel_lon'] = None
    
    ais_encounter_data[f'{i}_encounter_distance'] = None
    ais_encounter_data[f'{i}_encounter_sog'] = None
    ais_encounter_data[f'{i}_encounter_cog'] = None
    # ais_encounter_data[f'{i}_encounter_relative_bearing'] = None
    ais_encounter_data[f'{i}_encounter_time'] = None
    

In [157]:
print(ais_own_data.columns)
print(ais_other_data.columns)
print(ais_encounter_data.columns)

Index(['mmsi', 'lat', 'lon', 'sog', 'cog', 'short_voyage_id', 'unixtime',
       'position', 'new_short_voyage_id', 'cog_sin', 'cog_cos'],
      dtype='object')
Index(['mmsi', 'lat', 'lon', 'sog', 'cog', 'short_voyage_id', 'unixtime',
       'NDC_nm', 'In_TSS', 'Vessel_TSS'],
      dtype='object')
Index(['mmsi', 'lat', 'lon', 'sog', 'cog', 'short_voyage_id', 'unixtime',
       'position', 'new_short_voyage_id', 'cog_sin', 'cog_cos',
       '1_encounter_vessel_mmsi', '1_encounter_vessel_lat',
       '1_encounter_vessel_lon', '1_encounter_distance', '1_encounter_sog',
       '1_encounter_cog', '1_encounter_time', '2_encounter_vessel_mmsi',
       '2_encounter_vessel_lat', '2_encounter_vessel_lon',
       '2_encounter_distance', '2_encounter_sog', '2_encounter_cog',
       '2_encounter_time', '3_encounter_vessel_mmsi', '3_encounter_vessel_lat',
       '3_encounter_vessel_lon', '3_encounter_distance', '3_encounter_sog',
       '3_encounter_cog', '3_encounter_time', '4_encounter_vessel_mmsi

In [159]:
# Compute encounters for each vessel

def find_encounters(ais_own_data, ais_other_data, ais_encounter_df, encounter_distance_limit):
    encounter_df = pd.DataFrame()
    vessels = ais_own_data["mmsi"].unique()
#     print(vessels)
    count = 0
    ais_encounter_data = ais_encounter_df.copy()

    
    for vessel in vessels:
        # if count < 3:

        count+=1
        print(count)
        
        vessel_data = ais_own_data[ais_own_data["mmsi"] == vessel].sort_values("unixtime")
        other_vessels_data = ais_other_data[ais_other_data["mmsi"] != vessel]

        for index, row in vessel_data.iterrows():
            enc_count = 1
            # print(index)
            lat, lon = row[["lat", "lon"]]
            other_vessels_data_v1= pd.DataFrame()
            other_vessels_data_v1 = other_vessels_data[other_vessels_data['unixtime'] == row['unixtime']] 
            
            if not other_vessels_data_v1.empty:
                # print(other_vessels_data_v1)
                # print('go on')
                other_vessels_data_v1.loc[:,'distance'] = other_vessels_data_v1.apply(lambda x: haversine((lat, lon), (x['lat'], x['lon']), unit=Unit.KILOMETERS), axis=1)
                # other_vessels_data_v1['distance'] = haversine((lat, lon), (other_vessels_data_v1['lat'], other_vessels_data_v1['lon']), unit= Unit.KILOMETERS)

                other_vessels_data_v2 = other_vessels_data_v1[other_vessels_data_v1['distance'] < encounter_distance_limit]
                
                other_vessels_data_v2 = other_vessels_data_v2.sort_values('distance', ascending=True)


                if not other_vessels_data_v2.empty:
                    for enc_vessel, enc_vessel_data in other_vessels_data_v2.groupby("mmsi"):
                        while enc_count < 8:
                            # print(enc_vessel_data)
                            ais_encounter_data.at[index,f'{enc_count}_encounter_vessel_mmsi'] = enc_vessel_data['mmsi']
                            ais_encounter_data.at[index,f'{enc_count}_encounter_vessel_lat'] = enc_vessel_data['lat']
                            ais_encounter_data.at[index,f'{enc_count}_encounter_vessel_lon'] = enc_vessel_data['lon']
                            ais_encounter_data.at[index,f'{enc_count}_encounter_distance'] = enc_vessel_data['distance']
                            # print(enc_vessel_data['distance'])
                            ais_encounter_data.at[index,f'{enc_count}_encounter_sog'] = enc_vessel_data['sog']
                            ais_encounter_data.at[index,f'{enc_count}_encounter_cog'] = enc_vessel_data['cog']
                            # ais_encounter_data.loc[index,f'{enc_count}_encounter_relative_bearing'] = enc_vessel_data['sog']
                            ais_encounter_data.at[index,f'{enc_count}_encounter_time'] = enc_vessel_data['mmsi']
                            
                            enc_count += 1

                else:
                    continue
                        
            else:
                continue
                        

            
        # else:
        #     break

    return (ais_encounter_data)

        


In [161]:
encounter_distance_limit = 3  # 2 km, for example
# encounter_time_window = 30  # 15 minutes, for example *NOT NEEDED TO BE USED ANY MORE AS CAN EQUATE TIME (ALL FOLLOW A UNIFROM TIME DISTRIBUTION WITH OVERLAPPIG TIME PERIOD)

encounters = find_encounters(ais_own_data = ais_own_data, ais_other_data= ais_other_data, ais_encounter_df = ais_encounter_data,  encounter_distance_limit =encounter_distance_limit)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [183]:
encounters.to_csv(to_store_path)